<a href="https://colab.research.google.com/github/abhg86/LLM/blob/main/papier/Pythia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [26]:
import torch
import torch.nn as nn
from torch.optim import Adam
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

import datasets

from transformers import GPTNeoXForCausalLM, pipeline, AutoTokenizer
from transformers import pipeline as transformers_pipeline

import json
import numpy as np
import pandas as pd

In [4]:

# pipeline = pipeline(
#     "text-generation",
#     model = "EleutherAI/pythia-160m-deduped",
#     revision="step143000",
#     cache_dir="./pythia-160m-deduped/step143000"
#     )

# model = pipeline.model
# tokenizer = pipeline.tokenizer
# # model = GPTNeoXForCausalLM.from_pretrained(
# #   "EleutherAI/pythia-70m-deduped",
# #   revision="step143000",
# #   cache_dir="./pythia-70m-deduped/step143000",
# # )
# # tokenizer = AutoTokenizer.from_pretrained(
# #     "EleutherAI/pythia-70m-deduped",
# #     revision="step143000",
# #     cache_dir="./pythia-70m-deduped/step143000",
# #     )

# inputs = tokenizer("Paris is the capital of", return_tensors="pt")
# tokens = model.generate(**inputs, max_length = 50)
# tokenizer.decode(tokens[0])

In [13]:
class Steer(nn.Module):
  def __init__(self, lm_head, embed_dim, num_steers=2, rank=1000, init_var=1e-2, epsilon=1e-3):
    super().__init__()
    self.projector1 = nn.Linear(embed_dim, rank)
    self.projector2 = nn.Linear(rank, embed_dim)
    self.lm_head = lm_head
    self.rank = rank
    self.epsilon = epsilon
    self.num_steers = num_steers
    self.embed_dim = embed_dim
    self.steer_values = torch.zeros(num_steers)
    self.weight = self.weight()

  def set_values(self, steer_values):
    self.steer_values = steer_values

  def forward(self, x):
    delta = self.projector2(self.projector1(x) * self.steer_values.unsqueeze(1))
    return self.lm_head(x + self.epsilon * delta)

  def regularization_term(self):
    return torch.norm(self.projector1.weight) + torch.norm(self.projector2.weight)

  def state_dict(self, destination=None, prefix='', keep_vars=False):
    # Call the superclass's state_dict method to handle the destination argument
    state_dict_ = super().state_dict(destination, prefix, keep_vars)

    # Add your custom state to the dictionary
    state_dict_[prefix + 'projector1'] = self.projector1.state_dict()
    state_dict_[prefix + 'projector2'] = self.projector2.state_dict()
    return state_dict_
    return {"projector1": self.projector1, "projector2": self.projector2}

  def load_state_dict(self, state_dict):
    self.projector1 = state_dict["projector1"]
    self.projector2 = state_dict["projector2"]

  def weight(self):
    return [self.projector1.weight, self.projector2.weight]


In [14]:
def train(dataloader, model, steer, tokenizer, n_steps=1000, lr=1e-2, training_steer=0, num_steers=2, max_length=256, regularization=1e-6):
    data_iter = iter(dataloader)

    device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
    model.to(device)

    print("number of training steps:", n_steps)
    start_step = 0
    optimizer = Adam(model.parameters(), lr=lr)

    pbar = tqdm(range(start_step, n_steps))

    for step_i in pbar:
        batch = next(data_iter, None)
        if batch is None:
            data_iter = iter(dataloader)
            batch = next(data_iter, None)

        cur_batch_size = len(batch["text"])
        batch_stance = torch.Tensor(batch["label"]).to(device)
        batch_stance = batch_stance.unsqueeze(1)
        batch_text = batch["text"]
        tokenized = tokenizer(batch_text, padding=True, max_length=max_length, truncation=True, return_tensors="pt")
        input_ids = torch.LongTensor(tokenized["input_ids"]).to(device)

        optimizer.zero_grad()

        attention_mask = torch.LongTensor(tokenized["attention_mask"]).to(device)

        steer.set_values(torch.Tensor(batch["label"]).to(device))

        position_ids = torch.arange(0, input_ids.shape[1], dtype=torch.long, device=device)
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids)

        # print("inpu_ids : ", input_ids.shape)
        # print("attention_mask : ", attention_mask.shape)
        # print("position_ids : ", position_ids.shape)
        # print("batch_stance : ", batch_stance)

        output = model(input_ids=input_ids, attention_mask=attention_mask, position_ids=position_ids, labels=input_ids)
        loss = output.loss
        regularization_term = steer.regularization_term()
        (loss + regularization * regularization_term).backward()
        optimizer.step()


    torch.save([
        model.state_dict(),
        max(n_steps, start_step)
    ], "train.pt")



In [15]:
dataset_train = datasets.load_dataset("SetFit/sst5")["train"]
dataloader_train = DataLoader(dataset_train, batch_size=32, shuffle=True)

Repo card metadata block was not found. Setting CardData to empty.


In [16]:
pipeline = pipeline(
    "text-generation",
    model = "EleutherAI/pythia-160m-deduped",
    revision="step143000",
    cache_dir="./pythia-160m-deduped/step143000",
    device= "cuda:0" if torch.cuda.is_available() else "cpu"
    )


In [17]:

model = pipeline.model
tokenizer = pipeline.tokenizer
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
model.config.pad_token_id = model.config.eos_token_id

for param in model.parameters():
  param.requires_grad = False

steer = Steer(model.embed_out, model.config.hidden_size)
model.lm_head = steer

# vocab_size = len(tokenizer)
# model.resize_token_embeddings(vocab_size)



In [10]:
train(dataloader_train, model, steer, tokenizer)

number of training steps: 1000


  1%|          | 8/1000 [02:01<4:11:12, 15.19s/it]


KeyboardInterrupt: 

In [18]:
model.load_state_dict(torch.load("train.pt")[0])

<ipython-input-18-9840cc2c63ae>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("train.pt")[0])


FileNotFoundError: [Errno 2] No such file or directory: 'train.pt'

In [19]:
def generate(prompt_data, tokenizer, model, steer, steer_values=[-5,1],
             prompt_num=3, prompt_length=20, num_beams=1, num_beam_groups=1,
             do_sample=True, temperature=1, top_p=0.9):
    device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
    model.to(device)
    model.eval()

    for _prompt in tqdm(prompt_data):
        _prompt["generations"] = []
        prompt_text = _prompt["prompt"]["text"]
        token_length = tokenizer(prompt_text,return_tensors="pt")["input_ids"].shape[1]

        steer_values = torch.Tensor(list(map(float,steer_values))).to(device)
        steer.set_values(steer_values[None])
        for _i in range(prompt_num):
            with torch.no_grad():
                inputs = tokenizer(prompt_text, return_tensors="pt").to(device)
                output = model.generate(
                    **inputs,
                    max_length=token_length+prompt_length,
                    min_length=token_length+prompt_length,
                    num_beams=num_beams,
                    num_beam_groups=num_beam_groups,
                    do_sample=do_sample,
                    temperature=temperature,
                    top_p=top_p,
                    pad_token_id=tokenizer.eos_token_id
                )
                output = tokenizer.decode(output[0], skip_special_tokens=True)
                output = output[len(prompt_text):]
                _prompt["generations"].append({
                    "text": output
                })
    return prompt_data

In [32]:
sentiment = "positive"

# Requires prompts from the paper's github repo (https://github.com/Glaciohound/LM-Steer/blob/main/data/prompts/sentiment_prompts-10k/positive_prompts.jsonl#L2498)
with open(sentiment + "_prompts.jsonl", "r") as f:
  prompt_data = list(map(json.loads, f.readlines()))

prompt_data = prompt_data[:10]

generated_data = generate(prompt_data, tokenizer, model, steer, steer_values=[-5,1])

100%|██████████| 10/10 [00:46<00:00,  4.67s/it]


In [22]:
print(generated_data[np.random.randint(0,len(generated_data))])

{'md5_hash': '6ef1539c763591976580201fb2bb67c9', 'prompt': {'text': '"I am delighted to'}, 'continuation': {'text': 'have been working on this.'}, 'num_positive': 25, 'generations': [{'text': ' have a family at home for Christmas.”\n\n"And this is why we have such a busy'}, {'text': ' join your people and will send my regards to you. I have never heard of the "Crow'}, {'text': ' be able to tell you about our new project, “We’re looking for a new designer and'}]}


In [23]:
def eval(classifier, generations_df, sentiment_file=None, max_length=256):

    # score generations and write to sentiment.jsonl
    print("writing outputs to ", str(sentiment_file))
    if sentiment_file is not None:
        fo = open(sentiment_file, 'w')
    from pprint import pprint
    accuracies = []
    for i, row in tqdm(generations_df.iterrows(), total=len(generations_df.index), desc='Scoring generation sentiments'):
        prompt = row.prompt['text']
        generations = [gen['text'] for gen in row['generations']]
        sentences_for_prompt= []
        for gen in generations:
            sentences_for_prompt.append(f'{prompt}{gen}')

        positive_proportion = 0
        try:
            predictions_for_prompt = classifier(sentences_for_prompt,
                                                max_length=512)
        except IndexError: # sometimes the generation is too long?
            print("exception occured, please check")
            predictions_for_prompt = [{'label': "", 'score': float('nan')}] * len(sentences_for_prompt)
        # print(predictions_for_prompt)
        for prediction in predictions_for_prompt:
            positive_proportion += float(prediction["label"] == "POSITIVE")
        positive_proportion = positive_proportion / len(predictions_for_prompt)
        # print(positive_proportion)
        accuracies.append(positive_proportion)
        # input()
        # print(predictions_for_prompt)
        if sentiment_file is not None:
            for res in predictions_for_prompt:
                fo.write(json.dumps(res) + '\n')
    return np.nanmean(accuracies), np.std(accuracies)


In [28]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

classifier = transformers_pipeline('sentiment-analysis', device=device)
# classifier = pipeline(model='siebert/sentiment-roberta-large-english')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [33]:
accuracy, std = eval(classifier, pd.DataFrame(generated_data))
print("accuracy : ", accuracy)
print("std : ", std)

writing outputs to  None


Scoring generation sentiments: 100%|██████████| 10/10 [00:02<00:00,  3.85it/s]

[1.0, 0.6666666666666666, 1.0, 1.0, 1.0, 1.0, 0.6666666666666666, 1.0, 1.0, 1.0]
accuracy :  0.9333333333333332
std :  0.13333333333333336


In [34]:
print(generated_data[-2])

{'md5_hash': '51b3bac12acd688a5f2a309e3e6f8a82', 'prompt': {'text': 'Kat, in particular, is good company'}, 'continuation': {'text': ', with her spray-paint spells and human resolve.'}, 'num_positive': 25, 'generations': [{'text': '. But for the first time ever, it’s all part of a bigger plan to build a'}, {'text': ', with the addition of the latest in a series of events.\n\nI did find my own'}, {'text': '." She has been a regular guest at his church for several decades. He has also become a regular'}]}
